# EEG - Flow

## 4. ICA select and apply

Last edit: 14.04.2023 22:07
@anguyen

In [ ]:
import os
from datetime import datetime

from mne import pick_types, read_annotations
from mne.io import read_info, read_raw_fif
from mne.preprocessing import ICA, read_ica
from mne.viz import set_browser_backend

from eeg_flow.config import load_config
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files

%matplotlib qt
set_browser_backend('qt')

_, derivatives_folder_root, experimenter = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
participant = 19  # int
group       = 6  # int
task        = "oddball"  # str
run         = 1  # int
reviewer = "Arthur"
load_existing_review = False #if False, loads the fresh unreviewed ICA components of the reviewer indicated above
consensus = "no"

########################################################################################

derivatives_folder = get_folder(derivatives_folder_root, participant, group)
fname_stem = get_fname(participant, group, task, run)

# create derivatives ica plots subfolder
os.makedirs(derivatives_folder / fname_stem / "plots" / "ica", exist_ok=True)

# create locks
derivatives = [
    derivatives_folder / fname_stem / (fname_stem + "_step4_1st-review-"+ reviewer +"-ica.fif"),
    derivatives_folder / fname_stem / (fname_stem + "_step4_2nd-review-"+ reviewer +"-ica.fif"),
]

if consensus == "yes":
    derivatives.append(derivatives_folder / fname_stem / (fname_stem + "_step4_preprocessed-raw.fif"))

locks = lock_files(*derivatives)

# load previous steps
## load raw recording
raw = read_raw_fif(derivatives_folder / fname_stem / (fname_stem + "_step1_raw.fif"), preload=True)
## load following annots
info = read_info(derivatives_folder / fname_stem / (fname_stem + "_step2_info.fif"))
annot = read_annotations(derivatives_folder / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif"))

# merge info and annots into current raw
raw.info["bads"] = info["bads"]
raw.set_annotations(annot)

# load ICAs
if not load_existing_review:
    fname_ica1 = derivatives_folder / fname_stem / (fname_stem + "_step3_1st-ica.fif")
    fname_ica2 = derivatives_folder / fname_stem / (fname_stem + "_step3_2nd-ica.fif")
else:
    fname_ica1 = derivatives_folder / fname_stem / (fname_stem + "_step4_1st-review-"+ reviewer +"-ica.fif"),
    fname_ica2 = derivatives_folder / fname_stem / (fname_stem + "_step4_2nd-review-"+ reviewer +"-ica.fif"),

ica1 = read_ica(fname_ica1)
ica2 = read_ica(fname_ica2)

In [ ]:
# Filter to final BP (1, 40) Hz
raw.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

## 4.1 Annotate bad ICs from ICA1 for mastoids
 - At this stage, let's only focus on the mastoids. Look for:
 - heartbeat in the IC-time series
 - muscle/noise on the mastoids on the topographic map

In [ ]:
figs_ica_sources_mastoids = ica1.plot_sources(title=fname_stem+" | ICA1 sources Mastoids | "+reviewer, show=True, inst=raw)

In [ ]:
figs_ica_comp_mastoids = ica1.plot_components(title=fname_stem+" | ICA1 components Mastoids | "+reviewer, show=True, inst=raw)

In [ ]:
ica_folder = derivatives_folder / fname_stem / "plots" / "ica"
print("a")
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
print("b")
for i in range(len(figs_ica_comp_mastoids)):
    print(i)
    save_path = os.path.join(ica_folder, "allComponents_ICA1_" + reviewer + "_" + str(i+1) + "_" +timestampStr+".svg")
    figs_ica_comp_mastoids[i].savefig(save_path, transparent=True)

In [ ]:
#save ICA1 here instead?

In [ ]:
if consensus:

    #%%% Filter the mastoids, apply ICA, and keep the mastoids
    # Final step in the preparation of our future reference, we need to filter
    # those channels to the desired frequencies, and apply the ICA.

    raw_mastoids = raw.copy()
    raw_mastoids.filter(
        l_freq=0.5,
        h_freq=40.0,
        picks="eeg",
        method="fir",
        phase="zero-double",
        fir_window="hamming",
        fir_design="firwin",
        pad="edge",
    )
    ica1.apply(raw_mastoids)
    raw_mastoids.pick_channels(["M1", "M2"])

    # Trick MNE in thinking that a custom-ref has been applied
    with raw_mastoids.info._unlock():
        raw_mastoids.info["custom_ref_applied"] = FIFF.FIFFV_MNE_CUSTOM_REF_ON

    # At this stage, the reference have been denoised. We have in 'raw_mastoids'
    # the 2 mastoids M1 and M2 referenced to CPz. Now, let's clean the rest.

## 4.2 Annotate bad ICs from ICA2 for EEG
 - At this stage, let's only focus on the mastoids. Look for:
 - heartbeat in the IC-time series
 - muscle/noise on the mastoids on the topographic map


In [ ]:
# Visual inspection
figs_ica_sources = ica.plot_sources(title=fname_stem+" | ICA2 sources | "+reviewer, show=True, inst=raw)

In [ ]:
figs_ica_comp = ica.plot_components(title=fname_stem+" | ICA2 components | "+reviewer, show=True, inst=raw)

In [ ]:
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")
for i in range(len(figs_ica_comp)):
    save_path =  os.path.join(ica_folder, "allComponents_ICA2_" + reviewer + "_" + str(i+1) + "_" + timestampStr+".svg")
    figs_ica_comp[i].savefig(save_path, transparent = True)

In [ ]:
if consensus: 
    #%% Apply ICA
    # At this stage, we have an ICA decomposition with labeled components. Now, we
    # can apply it on the initial raw object, filtered between the final
    # frequencies.
    # But for this operation to be valid, it needs to be referenced as raw_ica_fit.

    assert sorted(raw_ica_fit.info["bads"]) == sorted(raw.info["bads"])
    raw.filter(
        l_freq=0.5,
        h_freq=40.0,
        picks="eeg",
        method="fir",
        phase="zero-double",
        fir_window="hamming",
        fir_design="firwin",
        pad="edge",
    )

    raw.set_montage(None)
    raw.add_reference_channels(ref_channels="CPz")
    raw.set_eeg_reference("average", projection=False)
    ica2.apply(raw)

    #%% Rereferenced to mastoids
    # At this stage, we have:
    # - raw_mastoids, the mastoids cleaned and referenced to CPz
    # - raw, the other electrodes, cleaned + bads and referenced to CAR

    raw.set_eeg_reference(["CPz"], projection=False)  # change reference back
    raw.add_channels([raw_mastoids])
    raw.set_montage("standard_1020")  # add montage for non-mastoids
    raw.set_eeg_reference(["M1", "M2"])

    # raw.interpolate_bads(reset_bads=True, mode='accurate')
    # raw.plot(title="ICA applied on raw")

    # Last visual inspection
    raw.plot(theme="light")


## 4.3 Save derivatives

The ICA decomposition can be saved.

In [ ]:
fname_ica1 = derivatives_folder / fname_stem / (fname_stem + "_step4-1st-reviewed-ica.fif")
fname_ica2 = derivatives_folder / fname_stem / (fname_stem + "_step4_2nd-reviewed-ica.fif")

ica1.save(fname_ica1, overwrite=False)
ica2.save(fname_ica2, overwrite=False)

if consensus:
    fname_raw = derivatives_folder / fname_stem / (fname_stem + "_step4_preprocessed-raw.fif")
    raw.save(fname_raw, overwrite=False)


Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway